In [132]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import importlib

import combined_analysis_bachelor.code.functions_for_pipeline as funcs
import combined_analysis_bachelor.code.movement_detection_functions as move_functions
import combined_analysis_bachelor.code.cnt_to_other_format as cnt_to_raw
import combined_analysis_bachelor.code.create_processed_files as process
from utils.get_sub_dir import get_sub_folder_dir

### 1. step: trim recording and run through processing pipeline ###

In [46]:
source_directory = "E:\external_val_cnt"
class_dir = get_sub_folder_dir("classification", "classification_data")
target_dir = f"{class_dir}/class_raw/external_val_data_raw2"

SUB = "class"
LOCS = {"BIP7": "brachioradialis_R",
        "BIP8": "deltoideus_R",
        "BIP3": "acc_x_hand_R",
        "BIP4": "acc_y_hand_R",
        "BIP5": "acc_z_hand_R",
        }
channels_c = ["BIP7", "BIP8","BIP3", "BIP4", "BIP5"]
EMG = ["BIP7", "BIP8"]
SF = 1000

project folder found: C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG
C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG\data\classification_data


In [47]:
cnt_to_raw.cnt_to_raw_hdf(SUB, source_directory, target_dir, channels_c, EMG, LOCS, SF)

retrieved data: ['E:\\external_val_cnt\\sub_class_test8.cnt', 'E:\\external_val_cnt\\sub_class_test9.cnt']
C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG\data\classification_data\sub-classification\EMG_ACC/class_raw/external_val_data_raw2\sub-class_recording_durations.json
Checking file: E:\external_val_cnt\sub_class_test8.cnt
Basename: sub_class_test8.cnt
Reading ANT file E:\external_val_cnt\sub_class_test8.cnt
Reading 0 ... 66405  =      0.000 ...    66.405 secs...
[27.352 63.131 63.134]
sub-class_test8.cnt
Saving to: C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG\data\classification_data\sub-classification\EMG_ACC/class_raw/external_val_data_raw2\sub-class_test8_raw.h5
Datei existiert: True
Checking file: E:\external_val_cnt\sub_class_test9.cnt
Basename: sub_class_test9.cnt
Reading ANT file E:\external_val_cnt\sub_class_test9.cnt


C:\Users\User\Documents\GitHub\lid_opm\combined_analysis_bachelor\code\cnt_to_other_format.py:45: RuntimeWarning: All EEG channels are not referenced to the same electrode.
  source = mne.io.read_raw_ant(filepath, preload=True)
C:\Users\User\Documents\GitHub\lid_opm\combined_analysis_bachelor\code\cnt_to_other_format.py:45: RuntimeWarning: Unit unit not recognized, not scaling. Please report the unit on a github issue on https://github.com/mne-tools/mne-python.
  source = mne.io.read_raw_ant(filepath, preload=True)


Reading 0 ... 125307  =      0.000 ...   125.307 secs...


C:\Users\User\Documents\GitHub\lid_opm\combined_analysis_bachelor\code\cnt_to_other_format.py:45: RuntimeWarning: All EEG channels are not referenced to the same electrode.
  source = mne.io.read_raw_ant(filepath, preload=True)


[ 75.464 120.482]
sub-class_test9.cnt
Saving to: C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG\data\classification_data\sub-classification\EMG_ACC/class_raw/external_val_data_raw2\sub-class_test9_raw.h5
Datei existiert: True

✅ Every File and .JSON processed and saved!


C:\Users\User\Documents\GitHub\lid_opm\combined_analysis_bachelor\code\cnt_to_other_format.py:45: RuntimeWarning: Unit unit not recognized, not scaling. Please report the unit on a github issue on https://github.com/mne-tools/mne-python.
  source = mne.io.read_raw_ant(filepath, preload=True)


In [101]:
source_dir1 = f"{class_dir}/class_raw/external_val_data_raw"
target_dir1 = f"{class_dir}/class_processed/external_val_data_proc"

source_dir2 = f"{class_dir}/class_raw/external_val_data_raw2"
target_dir2 = f"{class_dir}/class_processed/external_val_data_proc2"

In [99]:
importlib.reload(process)

<module 'combined_analysis_bachelor.code.create_processed_files' from 'C:\\Users\\User\\Documents\\GitHub\\lid_opm\\combined_analysis_bachelor\\code\\create_processed_files.py'>

In [97]:
#process.create_processed_files(source_dir, target_dir, [1,20], [20,450], SF)

In [143]:
external_test6 = pd.read_hdf(f"{target_dir1}/sub-class_test6_processed.h5", key="data")
external_test8 = pd.read_hdf(f"{target_dir2}/sub-class_test8_processed.h5", key="data")

In [75]:
plt.figure()
plt.plot(external_test6["Sync_Time (s)"], external_test6["SVM_R"])
plt.show()

In [144]:
external_test6 = funcs.crop_edges(external_test6)
external_test8 = funcs.crop_edges(external_test8)

dfs = [external_test6, external_test8]

In [173]:
external_test8["Sync_Time (s)"] = external_test8["Sync_Time (s)"] - external_test8["Sync_Time (s)"].iloc[0]

In [108]:
for df in dfs:
    df["SVM"] = df["SVM_R"]
    df.drop("SVM_R", axis=1, inplace=True)
EMG_cols = ["brachioradialis_R", "deltoideus_R"]
ACC_col = ["SVM", "acc_x_hand_R", "acc_y_hand_R", "acc_z_hand_R"] 

In [109]:
dfs_proc = []
for df in dfs:
    df_with_tkeo = funcs.add_tkeo_add_envelope(df, EMG_cols, 20, 3, 1000)
    df_processed = funcs.apply_savgol_rms_acc(df_with_tkeo, ["SVM", "acc_x_hand_R", "acc_y_hand_R", "acc_z_hand_R"], rms_window_size=100)
    df_proc = df_processed.dropna(how="any", ignore_index=True)
    df_proc["Sync_Time (s)"] = df_proc["Sync_Time (s)"] - df_proc["Sync_Time (s)"].iloc[0]
    
    fig, axs = plt.subplots(4, 1)
    axs = axs.ravel()
    axs[0].plot(df_proc["Sync_Time (s)"], df_proc["brachioradialis_R_tkeo_rms"], label="brachioradialis tkeo")
    axs[0].set_xlabel("time (s)")
    axs[0].legend()
    axs[1].plot(df_proc["Sync_Time (s)"], df_proc["deltoideus_R_tkeo_rms"], label="delt tkeo")
    axs[1].set_xlabel("time (s)")
    axs[1].legend()
    axs[2].plot(df_proc["Sync_Time (s)"], df_proc["SVM_smooth_rms"], label="SVM")
    axs[2].set_xlabel("time (s)")
    axs[2].legend()
    axs[3].plot(df_proc["Sync_Time (s)"], df_proc["acc_z_hand_R_smooth_rms"], label="acc_z_R")
    axs[3].set_xlabel("time (s)")
    axs[3].legend()
    plt.show()
    
    dfs_proc.append(df_proc)

In [171]:
importlib.reload(funcs)
importlib.reload(move_functions)

<module 'combined_analysis_bachelor.code.movement_detection_functions' from 'C:\\Users\\User\\Documents\\GitHub\\lid_opm\\combined_analysis_bachelor\\code\\movement_detection_functions.py'>

In [111]:
baselines = {"df_test6": [0,11],
             "df_test8": [0,7],
             }

df_test_proc6 = dfs_proc[0]
df_test_proc8 = dfs_proc[1]

channels = ["brachioradialis_R_tkeo_rms", "deltoideus_R_tkeo_rms", "SVM_smooth_rms"]

In [175]:
dict_onsets_offsets_test6 = funcs.simple_interactive_multichannel_plot(df_test_proc6, channels, baselines, "df_test6", 1000, give_out_dict=True)

In [176]:
dict_onsets_offsets_test8 = funcs.simple_interactive_multichannel_plot(df_test_proc8, channels, baselines, "df_test8", 1000, give_out_dict=True)

KeyboardInterrupt: 

In [141]:
dict_onsets_offsets_test8

{'brachioradialis_R_tkeo_rms': [[8449, 10849], [16745, 19866]],
 'deltoideus_R_tkeo_rms': [[23189, 24630], [27738, 29496]],
 'SVM_smooth_rms': [[22447, 25831], [27157, 30353]]}

In [136]:
dict_onsets_offsets_test6

{'brachioradialis_R_tkeo_rms': [[13815, 16316], [20087, 21065]],
 'deltoideus_R_tkeo_rms': [[25809, 28120], [32761, 35327]],
 'SVM_smooth_rms': [[15819, 16866], [25193, 29205], [32640, 35958]]}

In [74]:
# okay SVM sieht auch nicht zu gut aus, ich nehme z-Achse (dokumentieren, dass für sowas halt das machine learning von Vorteil wäre eig..) Oder halt bestimmte Instruktionen bei den Aufnahmen --> obwohl, ist bei der Suppression vllt garnicht so einfach zb die Hand recht grade zu halten oder?

In [85]:
# ich hab dann: 
# 1. on/offsets als dict raus
# was noch muss:
# zu binary arrays
# behavioral data raus kriegen. 

In [154]:
delt_activity_test8 = move_functions.fill_activity_mask(dict_onsets_offsets_test8["deltoideus_R_tkeo_rms"], 1000, external_test8["Sync_Time (s)"])
brachioradialis_activity_test8 = move_functions.fill_activity_mask(dict_onsets_offsets_test8["brachioradialis_R_tkeo_rms"], 1000, external_test8["Sync_Time (s)"])
acc_activity_test8 = move_functions.fill_activity_mask(dict_onsets_offsets_test8["SVM_smooth_rms"], 1000, external_test8["Sync_Time (s)"])

In [167]:
behavioral_labels = move_functions.create_behavioral_array_arm(delt_activity_test8, brachioradialis_activity_test8, acc_activity_test8)

In [168]:
behavioral_labels

array([0, 0, 0, ..., 0, 0, 0])

In [174]:
funcs.plot_with_behavior(external_test8, behavioral_labels, dict_onsets_offsets_test8["deltoideus_R_tkeo_rms"], dict_onsets_offsets_test8["brachioradialis_R_tkeo_rms"])

C:\Users\User\Documents\GitHub\lid_opm\combined_analysis_bachelor\code\functions_for_pipeline.py:901: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
